## Preparing summary dataframes for Tableau:

Tableau Public makes it difficult to connect to Postgresql as a Data Source. For my EDA/Summary Presentation, I need to produce a lot of summary DFs manually. Each will be loaded as a separate datasource. 

In this document, a summary DF is produced for each relevent Business Question.

In [1]:
#Standard Imports:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import sqlalchemy as sql

In [2]:
#Standard Settings:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
alt.renderers.enable('notebook')
alt.data_transformers.enable('default', max_rows=None)
%matplotlib inline 
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 1000)

In [75]:
#Locations
inputDir="./data/stage2/"
saveDir="./data/summaryEDA/"

In [42]:
#Load the dataset: 
ilsDF = pd.read_csv("./data/stage1/ILS_clean.csv")


In [22]:
#Join tables
vendorDF = pd.read_csv(inputDir+"vendornames.csv")
storeDF = pd.read_csv(inputDir+"storenames.csv")
countyDF = pd.read_csv(inputDir+"countynames.csv")
categoryDF = pd.read_csv(inputDir+"categorynames.csv")
itemDF = pd.read_csv(inputDir+"itemdescriptions.csv")

In [71]:
ilsDF.head()
itemDF.head()

,invoiceid,date,city,zipcode,storeid,countyid,categoryid,vendorid,itemid,bottlevolumeml,statebottlecost,statebottleretail,bottlessold,saleprice,volumesoldlitre,totalstatecost
0,S28865700001,11/09/2015,waterloo,50702,2538,7,1701100,962,238,1500,11.62,17.43,6,104.58,9,69.72
1,S29339300091,11/30/2015,muscatine,52761,2662,70,1701100,65,173,750,19.58,29.37,4,117.48,3,78.32
2,S28866900001,11/11/2015,holstein,51025,3650,47,1701100,962,238,1500,11.62,17.43,1,17.43,1,11.62
3,S29134300126,11/18/2015,onawa,51040,3723,67,1081200,305,258,6000,99.00,148.50,1,148.50,6,99.00
4,S29282800048,11/23/2015,pella,50219,2642,63,1701100,962,238,1500,11.62,17.43,6,104.58,9,69.72


,itemid,itemdescription
0,101,stolichnaya premium 80prf w2 stemless martini ...
1,102,bushmills 750ml wblackbush 50ml
2,104,broker's gin holiday w1 collins glass
3,107,bushmills black wrubber ice ball molds
4,108,midnight moon cran-apple pie


### Support Functions:

In [62]:
#Signature: String,List -> DataFrame
#Purpose: Group based on one column, select a subset of columns, and return summed group results in
#one dataframe.
def simplesumdf(groupCol,colList):
    df = ilsDF.groupby(groupCol,as_index=False)[colList].sum()
    return df

def geoinfo(df):
    df.insert(loc=1,column="state",value="iowa")
    df.insert(loc=2,column="country",value="usa")
    return df
    
#Signature: String,List,DataFrame,String -> DataFrame
#Purpose, first get summed dataframe by calling simplesumdf. Perform a table join and some column operations
#to get sums of columns for a particular descriptive column.
def mergesumdf(groupCol, colList, tableName, colSelect):
    summedDF = simplesumdf(groupCol,colList)
    summedDF = summedDF.merge(right=tableName,on=colSelect+"id",how="left")
    holdCol = summedDF[colSelect+"name"]
    summedDF.drop([colSelect+"id",colSelect+"name"],axis=1,inplace=True)
    summedDF.insert(loc=0,column=colSelect+"name",value=holdCol)
    return summedDF

### Maps:

For each geographic region, I want what the total retail cost of bottles was, and the sale price. We can calculate the profits in Tableau.


In [43]:
#Lets calculate retail cost by adding a new column.

ilsDF["totalstatecost"] = ilsDF["bottlessold"]*ilsDF["statebottlecost"] 
ilsDF.head()

,invoiceid,date,city,zipcode,storeid,countyid,categoryid,vendorid,itemid,bottlevolumeml,statebottlecost,statebottleretail,bottlessold,saleprice,volumesoldlitre,totalstatecost
0,S28865700001,11/09/2015,waterloo,50702,2538,7,1701100,962,238,1500,11.62,17.43,6,104.58,9,69.72
1,S29339300091,11/30/2015,muscatine,52761,2662,70,1701100,65,173,750,19.58,29.37,4,117.48,3,78.32
2,S28866900001,11/11/2015,holstein,51025,3650,47,1701100,962,238,1500,11.62,17.43,1,17.43,1,11.62
3,S29134300126,11/18/2015,onawa,51040,3723,67,1081200,305,258,6000,99.00,148.50,1,148.50,6,99.00
4,S29282800048,11/23/2015,pella,50219,2642,63,1701100,962,238,1500,11.62,17.43,6,104.58,9,69.72


**Which cities have the most number of sales? (map):**

In [80]:
citySumDF = simplesumdf("city",["totalstatecost","saleprice"])
citySumDF = geoinfo(citySumDF)
citySumDF.head()
citySumDF.to_csv(saveDir+"citySumDF.csv",index_label="id")
#citySumDF = ilsDF.groupby("city",as_index=False)[["totalstatecost","saleprice"]].sum()
#citySumDF.insert(loc=1,column="state",value="iowa")
#citySumDF.insert(loc=2,column="country",value="usa")


,city,state,country,totalstatecost,saleprice
0,ackley,iowa,usa,178885.31,312836.61
1,adair,iowa,usa,145786.79,241063.62
2,adel,iowa,usa,1044919.99,1702866.59
3,afton,iowa,usa,104328.20,185980.86
4,akron,iowa,usa,116535.74,173306.31


**Which counties have the most number of sales? (map):**

In [81]:
countySumDF = mergesumdf("countyid", ["totalstatecost","saleprice"], countyDF, "county")
countySumDF = geoinfo(countySumDF)
countySumDF.to_csv(saveDir+"countySumDF.csv",index_label="id")
countySumDF.head()

,countyname,state,country,totalstatecost,saleprice
0,Adair,iowa,usa,1386761.75,2274896.37
1,Adams,iowa,usa,333409.14,497661.38
2,Allamakee,iowa,usa,2704275.78,4503840.59
3,Appanoose,iowa,usa,2585417.97,4312569.37
4,Audubon,iowa,usa,560460.96,856103.24


**Which zipcodes have the most number of sales? (map):**

In [82]:
zipSumDF = simplesumdf("zipcode",["totalstatecost","saleprice"])
zipSumDF = geoinfo(zipSumDF)
zipSumDF.head()
zipSumDF.to_csv(saveDir+"zipSumDF.csv",index_label="id")


#zipSumDF = ilsDF.groupby("zipcode",as_index=False)[["totalstatecost","saleprice"]].sum()
#zipSumDF.insert(loc=1,column="state",value="iowa")
#zipSumDF.insert(loc=2,column="country",value="usa")
#zipSumDF.head()

,zipcode,state,country,totalstatecost,saleprice
0,50002,iowa,usa,145786.79,2.410636e+05
1,50003,iowa,usa,1044919.99,1.702867e+06
2,50005,iowa,usa,3980.71,1.333571e+04
3,50006,iowa,usa,325769.84,5.231327e+05
4,50009,iowa,usa,7182580.71,1.237176e+07


### Top Tens:

**Which stores have the most number of sales?**


In [83]:
bestStoresDF = mergesumdf("storeid", ["totalstatecost","saleprice"], storeDF, "store")
bestStoresDF.to_csv(saveDir+"bestStoresDF.csv",index_label="id")

**What catagories sold the most**

In [84]:
bestCategoriesDF = mergesumdf("categoryid", ["totalstatecost","saleprice"], categoryDF, "category")
bestCategoriesDF.to_csv(saveDir+"bestCategoriesDF.csv",index_label="id")

**Which vendors sold the most?**

In [85]:
bestVendorsDF = mergesumdf("vendorid", ["totalstatecost","saleprice"], vendorDF, "vendor").head()
bestVendorsDF.to_csv(saveDir+"bestVendorsDF.csv",index_label="id")

**Which items were the best selling**

In [87]:
#Sadly, our generalized code doesn't quite work for this one. Do it manually.
itemSumDF = simplesumdf("itemid",["totalstatecost","saleprice"])
itemSumDF = itemSumDF.merge(right=itemDF,on="itemid",how="left")
holdCol = itemSumDF["itemdescription"]
itemSumDF.drop(["itemid","itemdescription"],axis=1,inplace=True)
itemSumDF.insert(loc=0,column="itemdescription",value=holdCol)
itemSumDF.head()
itemSumDF.to_csv(saveDir+"bestItemsDF.csv",index_label="id")


,itemdescription,totalstatecost,saleprice
0,stolichnaya premium 80prf w2 stemless martini ...,899.10,3776.22
1,bushmills 750ml wblackbush 50ml,3354.00,5031.00
2,broker's gin holiday w1 collins glass,136.08,765.45
3,bushmills black wrubber ice ball molds,1044.50,2225.14
4,midnight moon cran-apple pie,1437.50,672.75


### Trend Lines and Distributions:

**What are the total Sales per month, for each year?**

**What is the distribution for sale prices?**
**What is the distribution for a derived column (profit)?**

In [88]:
#Lets get the distributions of prices. This will be a long, simple dataframe with just raw numbers:
distDF = ilsDF[["totalstatecost","saleprice"]]
distDF.head(10)

distDF.to_csv(saveDir+"distDF.csv",index_label="id")

,totalstatecost,saleprice
0,69.72,104.58
1,78.32,117.48
2,11.62,17.43
3,99.00,148.50
4,69.72,104.58
5,34.86,52.29
6,69.72,104.58
7,23.24,34.86
8,704.88,1057.32
9,139.44,209.16
